In [1]:
import tensorflow as tf
import seaborn as sns
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

OUT_PATH='/home/angel00540/work_/work/cgh_2020/paper_e1/tflite_model/DenseNet121.tflite'
SAVE_MODEL='DenseNet121_paper_e1_c4_p4_bs128_data_0115_2052.26-0.95-0.07.hdf5'
WEIGHT_PATH = '/home/angel00540/work_/work/cgh_2020/paper_e1/weight/'+SAVE_MODEL
TEST_PATH = '/home/angel00540/work_/work/cgh_2020/paper_e1/'

2022-03-10 23:10:31.793936: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
classes = []
labels = [ 'Velar', 'Stopping','Affricate','Consonant vowel']
for n, _, _ in os.walk(os.path.abspath(TEST_PATH+'/ori/')):
    classes.append(n.split('/')[-1])

classes = classes[1:]
classes = [int(x) for x in classes]
classes.sort()

classes = [str(x) for x in classes]

print(f'classes : {classes}')
print(f'label : {labels}')

classes : ['4', '6', '7', '14']
label : ['Velar', 'Stopping', 'Affricate', 'Consonant vowel']


In [3]:
feature_size=128
def mfcc_3ch_spectrum(wavfile):
    import librosa
    from skimage.transform import resize

    fix_n_sec = 2
    mini_sec =0.3
    num_channels = 3
    window_sizes = [25, 50, 100]
    hop_sizes = [10, 25, 50]
    y=[]
    try:
        y, sr = librosa.load(wavfile)
    except:
        print("read error: ",wavfile)

    if len(y)>0:
        # pad if len less 1 sec
        if len(y)/sr > fix_n_sec:
            x = y[:fix_n_sec * sr]
            run=1
        elif mini_sec < len(y)/sr < fix_n_sec :
            z = int(fix_n_sec*sr - len(y))
            if(z%2==0):
                z1 = int(z/2)
                z2 = int(z/2)
            else:
                z1 = int((z-1)/2)
                z2 = int(z1 + 1)
            try:
                x = np.pad(y,(z1,z2), 'linear_ramp', end_values=(0, 0))
                run=1
            except:
                print("pad error: ",wavfile)
            # ([0]*append_zero)+y+([0]*append_zero)
        elif len(y)/sr < mini_sec:
            print('len < '+mini_sec+': ',wavfile)
            run=0

        if run==1:
            # audio normalizedy
            normalizedy = librosa.util.normalize(x)

            specs = []
            for i in range(num_channels):

                window_length = int(round(window_sizes[i]*sr/1000))
                # print('win_len: ',window_length)
                hop_length = int(round(hop_sizes[i]*sr/1000))
                mel = librosa.feature.melspectrogram(
                    y=normalizedy, sr=sr,n_fft=window_length,hop_length=hop_length, win_length=window_length)
                mellog = np.log(mel + 1e-9)

                spec = librosa.util.normalize(mellog)

                spec = resize(mellog, (128, feature_size))
                spec = np.asarray(spec)
                specs.append(spec)

            # list to np array
            specs = np.asarray(specs)
            specs = np.moveaxis(specs,0,2)
            print('data.shape : ',specs.shape)
            return specs


In [6]:
IN_PATH = []
for root2, dirs2, files2 in os.walk(os.path.abspath('/home/angel00540/work_/work/cgh_2020/paper_e1/test_app_tf/data_2022.03.10.23.08.12/')):
    for file2 in files2:
        if(file2[-4:] == '.wav'):
            IN_PATH.append(os.path.join(root2, file2))

In [7]:
IN_PATH

['/home/angel00540/work_/work/cgh_2020/paper_e1/test_app_tf/data_2022.03.10.23.08.12/wordcard08_3.wav',
 '/home/angel00540/work_/work/cgh_2020/paper_e1/test_app_tf/data_2022.03.10.23.08.12/wordcard07_3.wav',
 '/home/angel00540/work_/work/cgh_2020/paper_e1/test_app_tf/data_2022.03.10.23.08.12/wordcard09_2.wav',
 '/home/angel00540/work_/work/cgh_2020/paper_e1/test_app_tf/data_2022.03.10.23.08.12/wordcard07_1.wav']

In [8]:
wav_npdata = []
dadsad = [ ]
for i in range(len(IN_PATH)):
    wav_file = IN_PATH[i]
    wav_file_name = wav_file.split("\\")[-1]
    print(f"wav_file_name : {wav_file_name}" )
    wav_mfcc = mfcc_3ch_spectrum(wav_file)
    print(f"wav_mfcc.shape : {wav_mfcc.shape}" )
    wav_npdata.append(wav_mfcc)
    dadsad.append(wav_file_name)
wav_npdata=np.asarray(wav_npdata)
print(f"\nwav_npdata.shape : {wav_npdata.shape}" )

wav_file_name : /home/angel00540/work_/work/cgh_2020/paper_e1/test_app_tf/data_2022.03.10.23.08.12/wordcard08_3.wav
data.shape :  (128, 128, 3)
wav_mfcc.shape : (128, 128, 3)
wav_file_name : /home/angel00540/work_/work/cgh_2020/paper_e1/test_app_tf/data_2022.03.10.23.08.12/wordcard07_3.wav
data.shape :  (128, 128, 3)
wav_mfcc.shape : (128, 128, 3)
wav_file_name : /home/angel00540/work_/work/cgh_2020/paper_e1/test_app_tf/data_2022.03.10.23.08.12/wordcard09_2.wav
data.shape :  (128, 128, 3)
wav_mfcc.shape : (128, 128, 3)
wav_file_name : /home/angel00540/work_/work/cgh_2020/paper_e1/test_app_tf/data_2022.03.10.23.08.12/wordcard07_1.wav
data.shape :  (128, 128, 3)
wav_mfcc.shape : (128, 128, 3)

wav_npdata.shape : (4, 128, 128, 3)


In [9]:
model = load_model(WEIGHT_PATH)
model.summary()

2022-03-10 23:12:25.044171: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-10 23:12:25.045495: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-10 23:12:25.098533: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3e:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-03-10 23:12:25.098570: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-03-10 23:12:25.478053: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-03-10 23:12:25.478158: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [10]:
test = model.predict(wav_npdata)
print(f"predict value: {test}")

2022-03-10 23:12:43.220213: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-10 23:12:43.275096: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz
2022-03-10 23:12:44.431438: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-03-10 23:12:44.740462: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


predict value: [[7.8245737e-02 1.1997670e-02 1.1293718e-04 9.0964371e-01]
 [1.1329230e-01 5.9433979e-01 1.8052384e-02 2.7431551e-01]
 [6.7234248e-01 4.4526956e-03 1.3810386e-03 3.2182378e-01]
 [2.8467607e-01 4.3408576e-04 7.1476563e-04 7.1417499e-01]]


In [8]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=OUT_PATH)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [9]:
ans = []
for i in range(len(wav_npdata)):
    tmp = tf.expand_dims(wav_npdata[i], axis=0)
   
    input_data = np.array(tmp, dtype=np.float32)
#     print(input_data)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    ans.append(output_data)
    print('.', end='')

...

In [10]:
ans

[array([[0.00802952, 0.4780019 , 0.00065998, 0.51330864]], dtype=float32),
 array([[2.4946763e-05, 1.0917346e-05, 4.5765992e-07, 9.9996364e-01]],
       dtype=float32),
 array([[3.8828084e-04, 5.3108436e-01, 2.6076668e-06, 4.6852478e-01]],
       dtype=float32)]